In [4]:
#Import statements
import pandas as pd
import numpy as np


In [5]:
in_dir = "../"
neg_vals = pd.read_csv(in_dir + "20231020_gap_fill_before_clean.csv")
neg_vals = neg_vals.dropna(subset="ID")

In [6]:
neg_vals_new = neg_vals.copy()
for c in neg_vals["ID"].unique():
    for s in neg_vals["Sector"].unique():
        temp_neg_vals = neg_vals[(neg_vals["ID"] == c) & (neg_vals["Sector"] == s)]
        add_row = temp_neg_vals[temp_neg_vals["Gas"] == 'co2']
        for yr in range(2015,2025):
            yr = str(yr)
            co2e = 0
            for gas, gwp in [("co2", 1), ("ch4", 27.9), ("n2o", 273)]:
                if gas in temp_neg_vals["Gas"].unique():
                    co2e += temp_neg_vals.loc[temp_neg_vals["Gas"] == gas, yr].values[0] * gwp
            add_row.loc[:, yr] = co2e
        add_row.loc[:, "Gas"] = 'co2e_100yr'
        neg_vals_new = pd.concat([neg_vals_new, add_row])
neg_vals_new.dropna(subset="ID", inplace=True)

/var/folders/m6/jjnlxlvd1cg27ft29fy02vcr0000gn/T/ipykernel_97120/1979582704.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_row.loc[:, yr] = co2e
/var/folders/m6/jjnlxlvd1cg27ft29fy02vcr0000gn/T/ipykernel_97120/1979582704.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_row.loc[:, "Gas"] = 'co2e_100yr'
/var/folders/m6/jjnlxlvd1cg27ft29fy02vcr0000gn/T/ipykernel_97120/1979582704.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [ ]:
# Filter for rows where "Gas" is "co2e_100yr" and values in the year columns are negative
neg_vals_filtered = neg_vals_new[
    (neg_vals_new["Gas"] == "co2e_100yr") & 
    (neg_vals_new[[str(i) for i in range(2015, 2024)]] < 0).any(axis=1)
]

# Set positive values to 0 to exclude them from the sum
neg_vals_filtered[[str(i) for i in range(2015, 2024)]] = neg_vals_filtered[[str(i) for i in range(2015, 2024)]].applymap(lambda x: x if x < 0 else 0)

# Group by "Sector" and sum
result = neg_vals_filtered.groupby("Sector")[[str(i) for i in range(2015, 2024)]].sum().T

result.to_csv("neg_val_sum.csv")

/var/folders/m6/jjnlxlvd1cg27ft29fy02vcr0000gn/T/ipykernel_97120/1581915556.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_vals_filtered[[str(i) for i in range(2015, 2024)]] = neg_vals_filtered[[str(i) for i in range(2015, 2024)]].applymap(lambda x: x if x < 0 else 0)


In [8]:
neg_counts = (neg_vals_filtered[[str(i) for i in range(2015, 2024)]] < 0).groupby(neg_vals_filtered["Sector"]).sum().T
neg_counts.to_csv("neg_val_count.csv")

In [ ]:
#Fraction of all implicitly derived sectors that are negative:
(neg_vals_new[[str(i) for i in range(2015, 2024)]]<0).sum().sum() / neg_vals_new[[str(i) for i in range(2015, 2024)]].count().sum()


0.06270095341716708